In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [12]:
#export

from exp.nb_03_early_stopping import *
torch.set_num_threads(2)

# ConvNet

In [13]:
x_train, y_train, x_valid, y_valid = get_data(url=MNIST_URL)

In [14]:
#export

def normalize_to(train, valid):
    mean = train.mean()
    std  = train.std()
    return normalize(train, mean, std), normalize(valid, mean, std)

In [15]:
x_train, x_valid = normalize_to(x_train, x_valid)
train_ds = Dataset(x_train, y_train)
valid_ds = Dataset(x_valid, y_valid)

In [16]:
### checking if the normalization worked properly or not
### i.e. mean ~= 0 & std ~= 1

x_train.mean(), x_train.std()

(tensor(3.0614e-05), tensor(1.))

In [17]:
nh = 50
bs = 512
c  = y_train.max().item() + 1
loss_func = F.cross_entropy

data = DataBunch(*get_dls(train_ds, valid_ds, bs=bs), c=c)

```
To refactor layers, it's useful to have a Lambda layer that can take a basic function and convert it to a layer we can put inside "torch.nn.Sequential"

NOTE: if you use a Lambda layer with a "lambda" function, your model won't pickle so you won't be save it with PyTorch. So it's best to give a name to the function you are using inside your Lambda (like "flatten" below).
```

In [19]:
#export

class Lambda(torch.nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x):
        return self.func(x)

def flatten(x):
    return x.view(x.shape[0], -1)

def mnist_resize(x):
    return x.view(-1, 1, 28, 28)

In [23]:
### We can now define a simple CNN

def get_cnn_model(data):
    return torch.nn.Sequential(
        Lambda(func=mnist_resize),                                                           ### 28x28
        torch.nn.Conv2d( 1, 8,  kernel_size=5, padding=2, stride=2), torch.nn.ReLU(),        ### 14x14
        torch.nn.Conv2d( 8, 16, kernel_size=3, padding=2, stride=2), torch.nn.ReLU(),        ### 7x7
        torch.nn.Conv2d(16, 32, kernel_size=3, padding=2, stride=2), torch.nn.ReLU(),        ### 4x4
        torch.nn.Conv2d(32, 32, kernel_size=3, padding=2, stride=2), torch.nn.ReLU(),        ### 2x2
        torch.nn.AdaptiveAvgPool2d(1),
        Lambda(func=flatten),
        torch.nn.Linear(32, data.c)
    )

In [24]:
model = get_cnn_model(data)

In [25]:
model

Sequential(
  (0): Lambda()
  (1): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (2): ReLU()
  (3): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (4): ReLU()
  (5): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (6): ReLU()
  (7): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (8): ReLU()
  (9): AdaptiveAvgPool2d(output_size=1)
  (10): Lambda()
  (11): Linear(in_features=32, out_features=10, bias=True)
)

In [29]:
### basic callbacks from previous notebook: imflash217__03_early_stopping.ipynb
cbfs = [Recorder, partial(AvgStatsCallback, accuracy)]

In [30]:
opt = optim.SGD(params=model.parameters(), lr=0.4)
learner = Learner(model=model, opt=opt, loss_fn=loss_func, data=data)
run = Runner(cb_funcs=cbfs)

In [31]:
%time run.fit(epochs=1, learner=learner)

train: [2.301403125, tensor(0.1106)]
valid: [2.3005251953125, tensor(0.1064)]
CPU times: user 12.7 s, sys: 3.64 s, total: 16.4 s
Wall time: 5.76 s


# `CUDA`:

```
A simple callback can make sure the model, inputs & outputs are on the same device.
```

In [32]:
### Somewhat more flexible way
device = torch.device("cuda", 0)

In [33]:
device

device(type='cuda', index=0)

In [34]:
class CudaCallback(Callback):
    def __init__(self, device):
        self.device = device
    def begin_fit(self):
        self.model.to(self.device)
    def begin_batch(self):
        self.run.xb = self.xb.to(self.device)
        self.run.yb = self.yb.to(self.device)

In [35]:
### Somewhat less flexible but more convenient
torch.cuda.set_device(device)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [36]:
#export

class CudaCallback(Callback):
    def begin_fit(self):
        self.model.cuda()
    def begin_batch(self):
        self.run.xb = self.xb.cuda()
        self.run.yb = self.yb.cuda()

In [38]:
cbfs.append(CudaCallback)

In [40]:
model = get_cnn_model(data=data)
opt = optim.SGD(params=model.parameters(), lr=0.4)
learner = Learner(model=model, opt=opt, loss_fn=loss_func, data=data)
run = Runner(cb_funcs=cbfs)

In [41]:
%time run.fit(epochs=1, learner=learner)

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx